In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
import numpy as np

In [2]:
train_dir = "animals"
test_dir = "animals"

In [3]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 800 images belonging to 2 classes.
Found 199 images belonging to 2 classes.
Found 999 images belonging to 2 classes.


In [5]:
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# CNN model with dropout and regularization
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.3),
    
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.4),
    
    layers.Flatten(),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Configure the model optimizers, loss function, and metrics
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

c:\Users\Justin\25-26\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator
)

Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.5188 - loss: 1.3978 - val_accuracy: 0.5025 - val_loss: 1.0810
Epoch 2/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.5238 - loss: 1.0185 - val_accuracy: 0.4975 - val_loss: 0.9576
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.5525 - loss: 0.9106 - val_accuracy: 0.6583 - val_loss: 0.8573
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.6575 - loss: 0.8170 - val_accuracy: 0.6935 - val_loss: 0.8154
Epoch 5/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.6575 - loss: 0.7636 - val_accuracy: 0.6482 - val_loss: 0.7558
Epoch 6/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.7100 - loss: 0.7037 - val_accuracy: 0.7638 - val_loss: 0.6647
Epoch 7/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.6850 - loss: 0.6761 - val_accuracy: 0.7337 - val_loss: 0.6192
Epoch 8/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 69s 3s/step - accuracy: 0.7412 - loss: 0.5997 - val_accuracy: 0.7990 - val_loss:

In [7]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 308ms/step - accuracy: 0.9259 - loss: 0.2256
Test Accuracy: 0.9259259104728699


In [8]:
model.save('cats_vs_dogs_cnn.h5')

In [9]:
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='cats_vs_dogs_cnn.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Dog" if pred >= 0.5 else "Cat"
    confidence = pred if pred >= 0.5 else 1 - pred
    print(f"Prediction: {label} (confidence: {confidence:.2%})")
    return label, confidence

In [10]:
predict_image("run/run_1a.png")
predict_image("run/run_1b.png")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step


Prediction: Dog (confidence: 89.95%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
Prediction: Cat (confidence: 98.93%)


('Cat', np.float32(0.9893367))